<a href="https://colab.research.google.com/github/LeonBecken/Bezkrovnyi_Assignments/blob/main/Bezkrovnyi_10_2_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 10.2

> Replace all TODOs with your code. Do not change any other code.

In [5]:
# Do not edit this cell

import csv
import unittest


## Clean code

### Task 1

You are given a function that reads a csv file with temperature measurements (see example below), converts Fahrenheit values to Celsius, calculates and prints some statistics, and writes to another file. It looks a bit messy, let's clean it up!

Example file:
```csv
Temperature (F)
78.5
81.2
75.9
82.1
```

Do the steps below one by one, editing the code in the cell:
1. Naming is so ambiguous and unclear, let's rename variables and function name with proper names.
2. Are these comments really useful?
3. This function does quite a lot, let's divide it in the way that each function does only one thing, and there's one main function that uses others.
4. There seem to be some magic coefficients in the temperature conversion part; let's make them obvious.

If you find any additional improvements, feel free to implement them and leave a comment under your code with an explanation.

In [6]:
# Константи для переведення температури з Фаренгейта у Цельсій
FAHRENHEIT_OFFSET = 32
FAHRENHEIT_TO_CELSIUS = 5 / 9


def read_fahrenheit_values(input_file):
    """Зчитує значення температур у Фаренгейтах з CSV і повертає їх у вигляді списку"""
    temperatures = []
    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Пропускаємо заголовок
        for row in reader:
            temperatures.append(float(row[0]))
    return temperatures


def convert_to_celsius(fahrenheit_values):
    """Конвертує список температур з Фаренгейта у Цельсій"""
    return [(f - FAHRENHEIT_OFFSET) * FAHRENHEIT_TO_CELSIUS for f in fahrenheit_values]


def calculate_statistics(values):
    """Обчислює середнє, мінімальне та максимальне значення зі списку чисел"""
    if not values:
        return None  # Якщо список порожній — повертаємо None
    return {
        'average': sum(values) / len(values),
        'min': min(values),
        'max': max(values)
    }


def write_celsius_to_csv(celsius_values, output_file):
    """Записує значення температур у Цельсіях у CSV-файл"""
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Temperature (C)'])  # Заголовок стовпця
        for value in celsius_values:
            writer.writerow([value])  # Кожне значення — окремий рядок


def process_temperature_data(input_file, output_file):
    """Головна функція: читає, конвертує, обчислює статистику та записує результат"""
    fahrenheit_values = read_fahrenheit_values(input_file)
    celsius_values = convert_to_celsius(fahrenheit_values)
    stats = calculate_statistics(celsius_values)

    print("Статистика температур:")
    print("Середня: {:.2f}°C".format(stats['average']))
    print("Мінімальна: {:.2f}°C".format(stats['min']))
    print("Максимальна: {:.2f}°C".format(stats['max']))

    write_celsius_to_csv(celsius_values, output_file)
    return stats

"""
Покращення, які були внесені:
1. Змінено імена функцій та змінних на більш зрозумілі.
2. Поділено велику функцію на кілька менших, кожна виконує одну дію.
3. Додано константи для конвертації температур (щоб уникнути "магічних чисел").
4. Додано докстрінги (короткі описи функцій).
5. Код став більш читабельним і легким для тестування та повторного використання
"""

'\nПокращення, які були внесені:\n1. Змінено імена функцій та змінних на більш зрозумілі.\n2. Поділено велику функцію на кілька менших, кожна виконує одну дію.\n3. Додано константи для конвертації температур (щоб уникнути "магічних чисел").\n4. Додано докстрінги (короткі описи функцій).\n5. Код став більш читабельним і легким для тестування та повторного використання\n'

### Task 2

How would you write tests for the initial implementation? What exactly would you test in the function?

I hope you see now that once functionality is separated, it's easier to test it in isolation. So, let's write a couple of unit tests for your function and one integration test for your main function.

Hint: you would probably want to mock reading from/writing to file to make the test independent from the environment.

In [9]:
import io
from unittest.mock import mock_open, patch, MagicMock

# Імпортуємо функції з модуля (заміни 'temp_module' на реальну назву файлу без .py)
# from temp_module import (
#     read_fahrenheit_values,
#     convert_to_celsius,
#     calculate_statistics,
#     write_celsius_to_csv,
#     process_temperature_data,
# )

class UnitTestCase(unittest.TestCase):
    """Юніт-тести: перевіряємо маленькі, ізольовані частини логіки."""

    def test_convert_to_celsius_simple(self):
        # Перевіряємо правильність формули конвертації
        from math import isclose
        from temp_module import convert_to_celsius  # імпортуємо з модуля

        fahrenheit = [32.0, 50.0, 68.0, 77.0]  # 0, 10, 20, 25 °C
        expected = [0.0, 10.0, 20.0, 25.0]
        result = convert_to_celsius(fahrenheit)
        self.assertEqual(len(result), len(expected))
        for r, e in zip(result, expected):
            self.assertTrue(isclose(r, e, rel_tol=1e-9, abs_tol=1e-9))

    def test_calculate_statistics_normal(self):
        # Перевіряємо середнє/мін/макс
        from temp_module import calculate_statistics

        values = [0.0, 10.0, 20.0, 25.0]
        stats = calculate_statistics(values)
        self.assertIsInstance(stats, dict)
        self.assertAlmostEqual(stats["average"], sum(values) / len(values), places=9)
        self.assertEqual(stats["min"], 0.0)
        self.assertEqual(stats["max"], 25.0)

    def test_calculate_statistics_empty(self):
        # Порожній список — повертаємо None (захист від помилок)
        from temp_module import calculate_statistics

        self.assertIsNone(calculate_statistics([]))

    def test_read_fahrenheit_values(self):
        # Мокуємо читання CSV: без залежності від файлової системи
        from temp_module import read_fahrenheit_values

        csv_data = "Temperature (F)\n78.5\n81.2\n75.9\n"
        with patch("builtins.open", mock_open(read_data=csv_data)):
            values = read_fahrenheit_values("input.csv")
        self.assertEqual(values, [78.5, 81.2, 75.9])

    def test_write_celsius_to_csv(self):
        # Перевіряємо запис: підміняємо csv.writer, щоб фіксувати рядки
        from temp_module import write_celsius_to_csv

        captured_rows = []

        class DummyWriter:
            def writerow(self, row):
                captured_rows.append(row)

        m = mock_open()
        with patch("builtins.open", m):
            with patch("csv.writer", return_value=DummyWriter()) as writer_mock:
                write_celsius_to_csv([0.0, 10.0, 20.0], "out.csv")

        # Перевіряємо, що спочатку заголовок, далі значення
        self.assertGreaterEqual(len(captured_rows), 4)
        self.assertEqual(captured_rows[0], ["Temperature (C)"])
        self.assertEqual(captured_rows[1], [0.0])
        self.assertEqual(captured_rows[2], [10.0])
        self.assertEqual(captured_rows[3], [20.0])


class IntegrationTestCase(unittest.TestCase):
    """Інтеграційний тест: перевіряємо, що весь пайплайн працює разом"""

    def test_process_temperature_data_end_to_end(self):
        """
        Будуємо «фейкове» вхідне CSV, перевіряємо:
        1) друк статистики у stdout
        2) повернуті stats
        3) рядки, які були б записані у вихідний CSV
        from temp_module import process_temperature_data
        """

        input_csv = "Temperature (F)\n78.5\n81.2\n75.9\n82.1\n"

        # Підміняємо open для читання і запису
        m = mock_open(read_data=input_csv)

        captured_rows = []

        class DummyWriter:
            def writerow(self, row):
                captured_rows.append(row)

        # Захоплюємо stdout, щоб перевірити надруковану статистику
        fake_stdout = io.StringIO()

        with patch("builtins.open", m):
            with patch("csv.writer", return_value=DummyWriter()):
                with patch("sys.stdout", fake_stdout):
                    stats = process_temperature_data("in.csv", "out.csv")

        # 1) Статистика роздрукована
        out = fake_stdout.getvalue()
        self.assertIn("Статистика температур:", out)
        self.assertIn("Середня:", out)
        self.assertIn("Мінімальна:", out)
        self.assertIn("Максимальна:", out)

        # 2) Статистика повернена як словник з потрібними ключами
        self.assertIsInstance(stats, dict)
        for k in ("average", "min", "max"):
            self.assertIn(k, stats)

        # 3) У вихідний CSV записано заголовок і всі конвертовані значення
        self.assertGreaterEqual(len(captured_rows), 1 + 4)
        self.assertEqual(captured_rows[0], ["Temperature (C)"])

        # Контрольне перетворення для порівняння
        expected_c = [ (78.5 - 32) * 5/9,
                       (81.2 - 32) * 5/9,
                       (75.9 - 32) * 5/9,
                       (82.1 - 32) * 5/9 ]

        # Значення, які були "записані"
        written_values = [row[0] for row in captured_rows[1:]]

        self.assertEqual(len(written_values), len(expected_c))
        for got, exp in zip(written_values, expected_c):
            self.assertAlmostEqual(got, exp, places=9)